In [1]:
#imports
import pandas as pd 
import numpy as np
import pickle
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.feature_extraction import text
import datetime
import ast

In [2]:
data = pd.read_csv('../../nowdata/parsing/overlaps_removed_df.csv', sep = "\t", low_memory=False, encoding="utf-8")

In [3]:
data['WEBTEXT'] = data['WEBTEXT'].apply(ast.literal_eval)

In [4]:
data.loc[0,"WEBTEXT"][2][3]

'icator Dashboard\nOur goal is to create 75,000 credentials/degrees by 2030. This can only happen when the entire community is purposeful through data in establishing and enabling conditions for long-term attainment success.\nCCI Dashboard Sponsored in part by the J.L. Bedsole Foundation\nHome\n | \nAbout\n | \nOur Work\n | \nOur Impact\n | \nNews/ Media\n | \nSupport\n | \nContact\n© 2018 Copyright Mobile Area Education Foundation.  All rights reserved. | Site by \n Web and New Media\n'

In [5]:
data['WEBTEXT'] = data['WEBTEXT'].fillna('0')

In [6]:
#webtext = data["WEBTEXT"]
#webtext_with_text = []
#for t in webtext:
#    try:
#        len_site = len(t)
#        for i in range(len_site):
#            webtext_with_text.append(t[i][3])
#    except:
#        pass

webtext = data["WEBTEXT"]
webtext_with_text = []
tokens = []
for t in webtext:
    try:
        len_site = len(t)
        #initializing a master string
        school_string = ''
        for i in range(len_site):
            school_string += t[i][3] + ' \n '
        webtext_with_text.append(school_string)
    except:
        pass

In [7]:
no_features = 1000
# creating custom stop words
my_additional_stop_word_list = []
#long list of manual addition to stopwords:
for i in range(1,13):
    my_additional_stop_word_list.append(datetime.date(2008, i, 1).strftime('%B'))
for i in range(1,13):
    my_additional_stop_word_list.append((datetime.date(2008, i, 1).strftime('%B')).lower())
for i in range(1, 2100):
    my_additional_stop_word_list.append(str(i))
my_additional_stop_word_list.append('00')
my_additional_stop_word_list.append('el')
my_additional_stop_word_list.append('en')
my_additional_stop_word_list.append('la')
my_additional_stop_word_list.append('los')
my_additional_stop_word_list.append('para')
my_additional_stop_word_list.append('las')
my_additional_stop_word_list.append('san')
my_additional_stop_word_list.append('mr')
my_additional_stop_word_list.append('mrs')
my_additional_stop_word_list.append('sa')
my_additional_stop_word_list.append('angeles')
my_additional_stop_word_list.append('diego')
my_additional_stop_word_list.append('california')
my_additional_stop_word_list.append('york')
my_additional_stop_word_list.append('fax')
my_additional_stop_word_list.append('email')
my_additional_stop_word_list.append('phone')
my_additional_stop_word_list.append('harlem')
my_additional_stop_word_list.append('bronx')
my_additional_stop_word_list.append('wi')
my_additional_stop_word_list.append('pm')
my_additional_stop_word_list.append('org')
my_additional_stop_word_list.append('city')
my_additional_stop_word_list.append('austin')
my_additional_stop_word_list.append('antonio')
my_additional_stop_word_list.append('texas')
#adding states abbreviations for stopwords
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV"]
for state in states:
          my_additional_stop_word_list.append(state)
for state in [state.lower() for state in states]:
          my_additional_stop_word_list.append(state)
stop_words_extra = text.ENGLISH_STOP_WORDS.union(my_additional_stop_word_list)
# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.5, min_df=0.02, max_features=no_features, stop_words=stop_words_extra)
tf = tf_vectorizer.fit_transform(webtext_with_text)
tf_feature_names = tf_vectorizer.get_feature_names()

In [8]:
# No. of topics - 20
no_topics = 20
# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='batch', learning_offset=50.,random_state=0).fit(tf)

/opt/conda/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [9]:
#old version
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic " + str(topic_idx) + ': ' + " ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
no_top_words = 20
display_topics(lda, tf_feature_names, no_top_words)
#topics to note: topic 1, topic 2, topic 4, topic , topic 15

Topic 0: idea schools college public regional act amended office suite title academy children teacher prep south campus services free principal national
Topic 1: board meeting click montessori schools value agenda child minutes children enrollment community newsletter office parent program day year read calendar
Topic 2: board charter pdf minutes meeting committee year report academy parents file program state parent plan agendas information schools financial shall
Topic 3: academies dallas story read reserved home rights copyright careers design organization support big non news alumni programs events web life
Topic 4: high life skills public charter notice info information news enrollment schools diploma career new board locations request apply program learn
Topic 5: schools learning charter online homeschool help stem dyslexia method summer curriculum service bullying problems class study funding data project home
Topic 6: academy success schools scholars read year scholar high new 

In [10]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

ModuleNotFoundError: No module named 'pyLDAvis'

## Try 2

In [ ]:
no_features = 1000
# creating custom stop words
my_additional_stop_word_list = []
#long list of manual addition to stopwords:
for i in range(1,13):
    my_additional_stop_word_list.append(datetime.date(2008, i, 1).strftime('%B'))
for i in range(1,13):
    my_additional_stop_word_list.append((datetime.date(2008, i, 1).strftime('%B')).lower())
for i in range(1, 2100):
    my_additional_stop_word_list.append(str(i))
my_additional_stop_word_list.append('00')
my_additional_stop_word_list.append('el')
my_additional_stop_word_list.append('en')
my_additional_stop_word_list.append('la')
my_additional_stop_word_list.append('los')
my_additional_stop_word_list.append('para')
my_additional_stop_word_list.append('las')
my_additional_stop_word_list.append('san')
my_additional_stop_word_list.append('mr')
my_additional_stop_word_list.append('mrs')
my_additional_stop_word_list.append('sa')
my_additional_stop_word_list.append('angeles')
my_additional_stop_word_list.append('diego')
my_additional_stop_word_list.append('california')
my_additional_stop_word_list.append('york')
my_additional_stop_word_list.append('fax')
my_additional_stop_word_list.append('email')
my_additional_stop_word_list.append('phone')
my_additional_stop_word_list.append('harlem')
my_additional_stop_word_list.append('bronx')
my_additional_stop_word_list.append('wi')
my_additional_stop_word_list.append('pm')
my_additional_stop_word_list.append('org')
my_additional_stop_word_list.append('city')
my_additional_stop_word_list.append('austin')
my_additional_stop_word_list.append('antonio')
my_additional_stop_word_list.append('texas')
#adding states abbreviations for stopwords
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV"]
for state in states:
          my_additional_stop_word_list.append(state)
for state in [state.lower() for state in states]:
          my_additional_stop_word_list.append(state)
stop_words_extra = text.ENGLISH_STOP_WORDS.union(my_additional_stop_word_list)
# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.5, min_df=0.02, max_features=no_features, stop_words=stop_words_extra)
tf = tf_vectorizer.fit_transform(webtext_with_text)
tf_feature_names = tf_vectorizer.get_feature_names()

In [ ]:
# No. of topics - 10
no_topics = 10
# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='batch', learning_offset=50.,random_state=0).fit(tf)

In [ ]:
#old version
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic " + str(topic_idx) + ': ' + " ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
no_top_words = 20
display_topics(lda, tf_feature_names, no_top_words)

In [ ]:
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)